# 1. Setup

In [2]:
%load_ext watermark

import warnings
from glob import glob
import numpy as np
import pandas as pd
import xarray as xr
from scipy.stats import linregress
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import cmocean.cm as cmo

%watermark -iv -co -v

Python implementation: CPython
Python version       : 3.12.10
IPython version      : 9.2.0

conda environment: cpl_ppe_co2

pandas    : 2.2.2
cartopy   : 0.24.1
cmocean   : 4.0.3
sys       : 3.12.10 | packaged by conda-forge | (main, Apr 10 2025, 22:21:13) [GCC 13.3.0]
matplotlib: 3.9.1
xarray    : 2025.4.0
numpy     : 2.0.1



In [3]:
xr.set_options(display_expand_attrs=False, display_expand_data=False);
plt.rcParams['figure.dpi'] = 120;

# CLM restart files

Variables of interest:
* canopy structure: htop, hbot, tlai
* carbon pools: leafc, 

In [4]:
init_dir = "/glade/derecho/scratch/bbuchovecky/coupPPE-hist/init"
branch_case = "b.e21.BHIST.f19_g17.CMIP6-historical-2deg.001"

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    clm_init = xr.open_dataset(f"{init_dir}/{branch_case}/1950-01-01/{branch_case}.clm2.r.1950-01-01-00000.nc")

In [5]:
key = "stemc"

for v in sorted(clm_init.variables):
    if key in v:
        print(v)

deadstemc
deadstemc_13
deadstemc_14
deadstemc_storage
deadstemc_storage_13
deadstemc_storage_14
deadstemc_xfer
deadstemc_xfer_13
deadstemc_xfer_14
livestemc
livestemc_13
livestemc_14
livestemc_storage
livestemc_storage_13
livestemc_storage_14
livestemc_to_litter
livestemc_xfer
livestemc_xfer_13
livestemc_xfer_14


# Differences in parameters between CMIP and coupPPE

In [6]:
cmip_pfile = xr.open_dataset('/glade/p/cesmdata/cseg/inputdata/lnd/clm2/paramdata/clm5_params.c171117.nc', decode_timedelta=False)
piso_pfile = xr.open_dataset('/glade/u/home/czarakas/coupled_PPE/data/paramfiles/OAAT0000.nc', decode_timedelta=False)

with open("lnd_in_CMIP6-AMIP-2deg", "r") as f:
    cmip_nml = f.read()

with open("lnd_in_coupPPE.000", "r") as f:
    piso_nml = f.read()

cmip_pname = set(cmip_pfile.variables)
piso_pname = set(piso_pfile.variables)

In [8]:
print(len(cmip_pname))
print(len(piso_pname))

print(len(piso_pname) - len(cmip_pname))

264
375
111


In [ ]:
cmip_not_in_piso = cmip_pname - piso_pname
print("params in cmip and not in piso:\n", cmip_not_in_piso)

params in cmip and not in piso:
 {'psi_soil_ref', 'k_nitr_max'}


In [ ]:
piso_not_in_cmip = piso_pname - cmip_pname
print("params in piso and not in cmip:\n", piso_not_in_cmip)

params in piso and not in cmip:
 {'watsat_adjustfactor', 'watsat_sf', 'zbedrock', 'k_nitr_max_perday', 'crit_onset_gdd_sf', 'jmaxse_sf', 'biofuel_harvfrac', 'snow_canopy_storage_scalar', 'relhExp', 'variantnames', 'wc2wjb0', 'cv', 'fnps', 'pd', 'theta_cj', 'kcha', 'act25', 'eta0_anderson', 'kc25_coef', 'liq_canopy_storage_scalar', 'taper', 'zbedrock_sf', 'wind_snowcompact_fact', 'lai_dl', 'clay_pf', 'csol_clay', 'jmaxb0', 'snowcan_unload_temp_fact', 'csoilc', 'rho_max', 'slopebeta', 'tpu25ratio', 'tpuha', 'zlnd', 'd_max', 'koha', 'sucsat_sf', 'jmaxhd', 'nstem', 'rswf_min', 'lmrhd', 'snw_rds_refrz', 'scvng_fct_mlt_sf', 'tkd_sand', 'fbw', 'theta_psii', 'pc', 'perched_baseflow_scalar', 'drift_gs', 'denitrif_respiration_exponent', 'hksat_sf', 'theta_ip', 'tkd_om', 'cpha', 'denitrif_nitrateconc_exponent', 'rswf_max', 'cp25_yr2000', 'season_decid_temperate', 'jmaxha', 'eta0_vionnet', 'ssi', 'denitrif_respiration_coefficient', 'dbh', 'aq_sp_yield_min', 'ko25_coef', 'wood_density', 'a_exp', 'v

In [25]:
for p in piso_not_in_cmip:
    if p in piso_nml:
        print(p, "in piso nml")
    if p in cmip_nml:
        print(p, "in cmip nml")

pd in piso nml
pd in cmip nml
pc in piso nml
pc in cmip nml
mu in piso nml
mu in cmip nml


# Differences in SSTs / boundary conditions between CMIP and coupPPE